<a href="https://colab.research.google.com/github/abinashp437/Stance_Detection_FNC_1/blob/main/lstm_fnc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords') #for stopword removal
nltk.download('wordnet')
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow import keras
from google.colab import files


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
body_url = "https://raw.githubusercontent.com/abinashp437/Stance_Detection_FNC_1/main/fnc_1_data/train_bodies.csv"
body = pd.read_csv(body_url)
stance_url = "https://raw.githubusercontent.com/abinashp437/Stance_Detection_FNC_1/main/fnc_1_data/train_stances.csv"
stance = pd.read_csv(stance_url)

In [3]:
for row_id, art in body.iterrows():
  body['articleBody'][row_id] = art['articleBody'].replace('\n','')
for row_id, stan in stance.iterrows():
  stance['Headline'][row_id] = stan['Headline'].replace('\n','')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [4]:
stop_words = stopwords.words('english')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
for row_id, art in body.iterrows():
  body['articleBody'][row_id] = ' '.join([word for word in art['articleBody'].split() if word not in stop_words])
for row_id, head in stance.iterrows():
  stance['Headline'][row_id] = ' '.join([word for word in head['Headline'].split() if word not in stop_words])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize(text):
  return [lemmatizer.lemmatize(word) for word in w_tokenizer.tokenize(text)]

for row_id, art in body.iterrows():
  body['articleBody'][row_id] = ' '.join(lemmatize(art['articleBody']))

for row_id, head in stance.iterrows():
  stance['Headline'][row_id] = ' '.join(lemmatize(head['Headline']))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [6]:
for row_id, art in body.iterrows():
  body['articleBody'][row_id] = art['articleBody'].lower()

for row_id, head in stance.iterrows():
  stance['Headline'][row_id] = head['Headline'].lower()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [7]:
corpus = []
for row_id, art in body.iterrows():
  corpus.append(w_tokenizer.tokenize(art['articleBody']))

for row_id, head in stance.iterrows():
  corpus.append(w_tokenizer.tokenize(head['Headline']))

In [8]:
pad_art = 180
pad_head = 11
for row_id, art in body.iterrows():
  body['articleBody'][row_id] = ' '.join(w_tokenizer.tokenize(art['articleBody'])[:pad_art])

for row_id, head in stance.iterrows():
  stance['Headline'][row_id] = ' '.join(w_tokenizer.tokenize(head['Headline'])[:pad_head])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:
model = Word2Vec(corpus, min_count = 1, size = 100, workers = 3, window = 3, iter = 30)
def vectorize(text):
  return [model[word] for word in w_tokenizer.tokenize(text)]

art_vec = pd.Series([], dtype = float)
for row_id, art in body.iterrows():
  art_vec[row_id] = vectorize(art['articleBody'])

head_vec = pd.Series([], dtype = float)
for row_id, head in stance.iterrows():
  head_vec[row_id] = vectorize(head['Headline'])

pad_art_vec = keras.preprocessing.sequence.pad_sequences(art_vec, padding = 'post', maxlen = pad_art, dtype = float)
pad_head_vec = keras.preprocessing.sequence.pad_sequences(head_vec, padding = 'post', maxlen = pad_head, dtype = float)
pad_art_vec = pad_art_vec.tolist()
pad_head_vec = pad_head_vec.tolist()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
article_vector = pd.DataFrame()
article_vector.insert(0, 'Body ID', body['Body ID'])
article_vector.insert(1, 'art_Vectors', pad_art_vec)

headline_vector = pd.DataFrame()
headline_vector.insert(0, 'Body ID', stance['Body ID'])
headline_vector.insert(1, 'head_Vectors', pad_head_vec)

vectors = pd.merge(article_vector, headline_vector, left_on = 'Body ID', right_on = 'Body ID')

**Training and Testing**

In [11]:
from tensorflow.keras.layers import Bidirectional, Dense, LSTM
from tensorflow.keras.models import model_from_json

In [12]:
Stan_Id = pd.Series([], dtype = int)
for row_id, stan in stance.iterrows():
  if stan['Stance'] == 'unrelated':
    Stan_Id[row_id] = 0
  elif stan['Stance'] == 'agree':
    Stan_Id[row_id] = 1
  elif stan['Stance'] == 'disagree':
    Stan_Id[row_id] = 2
  elif stan['Stance'] == 'discuss':
    Stan_Id[row_id] = 3

vectors.insert(3, 'Stance ID', Stan_Id)

In [13]:
vectors.head

<bound method NDFrame.head of        Body ID  ... Stance ID
0            0  ...         0
1            0  ...         1
2            0  ...         0
3            0  ...         0
4            0  ...         2
...        ...  ...       ...
49967     2532  ...         0
49968     2532  ...         0
49969     2532  ...         1
49970     2532  ...         3
49971     2532  ...         0

[49972 rows x 4 columns]>

In [14]:
temp_df = vectors.iloc[0:50]

In [15]:
def train(vector):
  x = pd.DataFrame()
  y = pd.DataFrame()
  x.insert(0, 'head', temp_df['head_Vectors'])
  x.insert(1, 'art', temp_df['art_Vectors'])
  y.insert(0, 'Stan_ID', temp_df['Stance ID'])
  return x, y

In [16]:
x_train, y_train = train(vectors)

In [17]:
print(len(x_train))
print(len(y_train))

50
50


In [18]:
def train_lstm(x_train, y_train):
    model_lstm = keras.Sequential()

    model_lstm.add(Bidirectional(LSTM(50, dropout = 0.5)))
    model_lstm.add(Dense(24,activation='sigmoid'))
    model_lstm.add(Dense(6,activation='softmax'))

    model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    fitmodel = model_lstm.fit(x_train, y_train, epochs = 15,  verbose = 1, shuffle=False)

In [19]:
x_train_dl = tf.convert_to_tensor(x_train)
y_train_dl = tf.convert_to_tensor(y_train)
acc_lstm_list = []

for n in [round(_ * 0.1, 1) for _ in range(2, 9)]:
  train_lstm(x_train_dl, y_train_dl, input_shape = (50,2))
  # acc_lstm_list.append(lstm_acc)

ValueError: ignored

In [ ]:
model_lstm.save('lstm_model.h5')